In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm
import os

C:\Users\ACER\AppData\Local\Temp\ipykernel_20568\99135878.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load RoBERTa pre-trained model and tokenizer for sentiment analysis
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

C:\Users\ACER\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Function to compute sentiment scores using RoBERTa
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return {
        'positive': scores[2],
        'neutral': scores[1],
        'negative': scores[0]
    }

# Function to process an Excel file and calculate sentiment percentages
def process_excel_file(file_path):
    # Load the Excel file
    df = pd.read_excel(file_path)
    
    # Check if 'Comments' column exists
    if 'Comments' not in df.columns:
        raise ValueError(f"'Comments' column not found in {file_path}")
    
    # Drop rows with missing values in the 'Comments' column
    comments = df['Comments'].dropna().tolist()
    
    # Initialize counters for sentiments
    total_comments = len(comments)
    pos_count, neu_count, neg_count = 0, 0, 0
    
    # Process each comment and calculate sentiment
    for comment in tqdm(comments, desc=f"Processing {file_path}"):
        try:
            sentiment_scores = polarity_scores_roberta(comment)
            if sentiment_scores['positive'] > sentiment_scores['neutral'] and sentiment_scores['positive'] > sentiment_scores['negative']:
                pos_count += 1
            elif sentiment_scores['neutral'] > sentiment_scores['positive'] and sentiment_scores['neutral'] > sentiment_scores['negative']:
                neu_count += 1
            else:
                neg_count += 1
        except Exception as e:
            print(f"Error processing comment: {e}")
    
    # Calculate sentiment percentages
    pos_percentage = (pos_count / total_comments) * 100
    neu_percentage = (neu_count / total_comments) * 100
    neg_percentage = (neg_count / total_comments) * 100
    
    return {
        'positive': pos_percentage,
        'neutral': neu_percentage,
        'negative': neg_percentage
    }


In [7]:
# List of Excel files to process
files = ['anura_cleaned.xlsx', 'namal_cleaned.xlsx', 'Ranil_cleaned.xlsx', 'sajith_cleaned.xlsx']

# Process each file and display results
for file in files:
    try:
        sentiment_percentages = process_excel_file(file)
        print(f"\nSentiment percentages for {file}:")
        print(f"Positive: {sentiment_percentages['positive']:.2f}%")
        print(f"Neutral: {sentiment_percentages['neutral']:.2f}%")
        print(f"Negative: {sentiment_percentages['negative']:.2f}%")
    except Exception as e:
        print(f"Error processing {file}: {e}")

Processing anura_cleaned.xlsx:   7%|▋         | 36/552 [00:03<00:33, 15.23it/s]

Error processing comment: The expanded size of the tensor (787) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 787].  Tensor sizes: [1, 514]


Processing anura_cleaned.xlsx: 100%|██████████| 552/552 [00:40<00:00, 13.73it/s]



Sentiment percentages for anura_cleaned.xlsx:
Positive: 51.81%
Neutral: 41.85%
Negative: 6.16%


Processing namal_cleaned.xlsx: 100%|██████████| 1663/1663 [01:48<00:00, 15.37it/s]


...


Sentiment percentages for Ranil_cleaned.xlsx:
Positive: 44.56%
Neutral: 47.69%
Negative: 7.75%


Processing sajith_cleaned.xlsx:  94%|█████████▍| 372/395 [00:26<00:01, 17.04it/s]

Error processing comment: The expanded size of the tensor (896) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 896].  Tensor sizes: [1, 514]


Processing sajith_cleaned.xlsx: 100%|██████████| 395/395 [00:28<00:00, 13.84it/s]


Sentiment percentages for sajith_cleaned.xlsx:
Positive: 21.27%
Neutral: 64.56%
Negative: 13.92%
